In [1]:
pip install pandas numpy scikit-learn beautifulsoup4 lxml tldextract python-dateutil joblib matplotlib seaborn language-tool-python 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import tarfile
import mailbox
import email
from email import policy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import language_tool_python
from urllib.parse import urlparse
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib  


d:\PhishGuard-AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Constants
SHORTENER_DOMAINS = {"bit.ly", "tinyurl.com", "t.co", "goo.gl", "ow.ly", "is.gd", "buff.ly"}
SUSPICIOUS_EXTS = {'.exe', '.scr', '.pif', '.jar', '.js', '.vbs', '.html', '.hta', '.docm', '.xlsm', '.xls', '.pptm', '.zip', '.rar'}

# Extract text parts and attachments
def get_text_parts(msg):
    txt_parts = []
    html_parts = []
    attachments = []
    for part in msg.walk():
        ctype = part.get_content_type()
        fn = part.get_filename()
        if fn:
            attachments.append(fn)
        if ctype == "text/plain" and not fn:
            try:
                txt_parts.append(part.get_payload(decode=True).decode(errors='ignore'))
            except:
                pass
        elif ctype == "text/html" and not fn:
            try:
                html_parts.append(part.get_payload(decode=True).decode(errors='ignore'))
            except:
                pass
    if not txt_parts and not html_parts:
        try:
            payload = msg.get_payload(decode=True)
            if isinstance(payload, bytes):
                txt_parts.append(payload.decode(errors='ignore'))
        except:
            pass
    return "\n".join(txt_parts), "\n".join(html_parts), attachments

# URL extraction
URL_REGEX = re.compile(r"https?://[^\s'\"<>]+", re.IGNORECASE)
IP_IN_URL = re.compile(r"https?://(?:\d{1,3}\.){3}\d{1,3}")

def extract_urls_from_html(html):
    urls = []
    anchors = []
    soup = BeautifulSoup(html, "lxml")
    for a in soup.find_all("a"):
        href = a.get('href')
        text = (a.get_text() or "").strip()
        if href:
            urls.append(href)
            anchors.append((href, text))
    for m in URL_REGEX.finditer(html):
        urls.append(m.group(0))
    return urls, anchors

# Parse mbox files
def parse_mbox(mbox_path, label, limit=None):
    m = mailbox.mbox(mbox_path)
    rows = []
    for i, msg in enumerate(m):
        if limit and i >= limit: break
        try:
            raw = msg.as_string()
            parsed = email.message_from_string(raw, policy=policy.default)
            subj = parsed.get("subject", "") or ""
            frm = parsed.get("from", "") or ""
            retpath = parsed.get("return-path", "") or parsed.get("Return-Path", "") or ""
            date = parsed.get("date", "") or ""
            try:
                date_parsed = dateparser.parse(date) if date else None
            except:
                date_parsed = None
            body_txt, body_html, attachments = get_text_parts(parsed)
            urls, anchors = extract_urls_from_html(body_html or body_txt)
            rows.append({
                "subject": subj,
                "from": frm,
                "return_path": retpath,
                "date": date_parsed,
                "body_text": body_txt,
                "body_html": body_html,
                "urls": urls,
                "anchors": anchors,
                "attachments": attachments,
                "label": label
            })
        except LookupError as e:
            print(f"Skipping email {i} in {mbox_path} due to encoding error: {e}")
            continue
        except Exception as e:
            print(f"Skipping email {i} in {mbox_path} due to general error: {e}")
            continue
    return pd.DataFrame(rows)

# Parse raw directory files
def parse_raw_dir(dir_path, label, limit=None):
    rows = []
    files = []
    for root, _, filenames in os.walk(dir_path):
        for fn in filenames:
            files.append(os.path.join(root, fn))
    for i, path in enumerate(files):
        if limit and i >= limit: break
        try:
            with open(path, "rb") as f:
                msg = email.message_from_binary_file(f, policy=policy.default)
        except:
            try:
                with open(path, "r", errors="ignore") as f:
                    msg = email.message_from_string(f.read(), policy=policy.default)
            except:
                continue
        subj = msg.get("subject", "") or ""
        frm = msg.get("from", "") or ""
        retpath = msg.get("return-path", "") or msg.get("Return-Path", "") or ""
        date = msg.get("date", "") or ""
        try:
            date_parsed = dateparser.parse(date) if date else None
        except:
            date_parsed = None
        body_txt, body_html, attachments = get_text_parts(msg)
        urls, anchors = extract_urls_from_html(body_html or body_txt)
        rows.append({
            "subject": subj,
            "from": frm,
            "return_path": retpath,
            "date": date_parsed,
            "body_text": body_txt,
            "body_html": body_html,
            "urls": urls,
            "anchors": anchors,
            "attachments": attachments,
            "label": label
        })
    return pd.DataFrame(rows)

In [6]:
# Load SpamAssassin ham (0)
df_ham = parse_raw_dir(r"D:\data\20021010_easy_ham", label=0)

# Load SpamAssassin spam (0 as well, since it's "non-phishing spam")
df_spam = parse_raw_dir(r"D:\data\20030228_spam", label=0)

# Load Nazario phishing (1)
df_phish1 = parse_mbox(r"D:\data\phishing1.mbox", label=1)
df_phish2 = parse_mbox(r"D:\data\phishing2.mbox", label=1)

# Combine
df = pd.concat([df_ham, df_spam, df_phish1, df_phish2], ignore_index=True)
print(df['label'].value_counts())




Skipping email 1318 in D:\data\phishing2.mbox due to encoding error: unknown encoding: iso-18899997-1
label
0    3052
1    1878
Name: count, dtype: int64


In [7]:
import tldextract
KEYWORDS = ["verify","password","account","urgent","click here","bank","login","update","suspend","confirm","security","ssn","credit card"]

def domain_of_address(addr):
    # extract registered domain from an email "Name <a@b.com>"
    if not addr:
        return ""
    # naive extract of email portion
    m = re.search(r"[\w\.-]+@([\w\.-]+)", addr)
    if m:
        dom = m.group(1)
    else:
        # maybe domain present directly
        dom = addr.strip()
    return tldextract.extract(dom).registered_domain or dom

def url_domain(u):
    try:
        return tldextract.extract(u).registered_domain
    except:
        return ""

def extract_features_row(r):
    subj = r.get("subject","") or ""
    body = r.get("body_text","") or ""
    fulltext = (subj + " " + body).strip()
    urls = r.get("urls") or []
    anchors = r.get("anchors") or []
    attachments = r.get("attachments") or []

    # Lexical
    subj_len = len(subj)
    subj_words = len(subj.split())
    body_len = len(body)
    body_words = len(body.split())
    letters = sum(1 for c in fulltext if c.isalpha())
    uppercase = sum(1 for c in fulltext if c.isupper())
    uppercase_ratio = uppercase / max(1, letters)
    exclaim_count = fulltext.count("!")
    question_count = fulltext.count("?")

    # keywords
    kw_counts = {f"kw_{k}": fulltext.lower().count(k) for k in KEYWORDS}

    # URL features
    num_urls = len(urls)
    num_ip_urls = sum(1 for u in urls if IP_IN_URL.search(u))
    num_shorteners = sum(1 for u in urls if urlparse(u).netloc.replace("www.","") in SHORTENER_DOMAINS)
    avg_url_len = np.mean([len(u) for u in urls]) if urls else 0
    unique_domains = {url_domain(u) for u in urls if url_domain(u)}
    num_unique_domains = len(unique_domains)

    # anchor mismatch: compare anchor text domain vs href domain (if anchor text contains a domain-like token)
    domain_mismatch = False
    for href, text in anchors:
        tdom = None
        hdom = url_domain(href)
        # rough parse anchor text looking for domain
        m = re.search(r"([\w\.-]+\.[a-z]{2,})", text)
        if m:
            tdom = tldextract.extract(m.group(1)).registered_domain
        if tdom and hdom and tdom != hdom:
            domain_mismatch = True
            break

    # header features
# header features
    from_domain = domain_of_address(r.get("from",""))
    ret_domain = domain_of_address(r.get("return_path",""))
    from_return_mismatch = bool(from_domain and ret_domain and from_domain != ret_domain)


    auth_headers = r.get("body_html","") + r.get("body_text","")
    # crude presence check (not authoritative)
    has_spf = bool(re.search(r"spf", auth_headers, re.I))
    has_dkim = bool(re.search(r"dkim", auth_headers, re.I))

    # attachments
    num_attachments = len(attachments)
    has_suspicious_attachment = any(os.path.splitext(fn)[1].lower() in SUSPICIOUS_EXTS for fn in attachments)

    feats = dict(
        subj_len=subj_len, subj_words=subj_words,
        body_len=body_len, body_words=body_words,
        uppercase_ratio=uppercase_ratio,
        exclaim_count=exclaim_count, question_count=question_count,
        num_urls=num_urls, num_ip_urls=num_ip_urls,
        num_shorteners=num_shorteners, avg_url_len=avg_url_len,
        num_unique_domains=num_unique_domains,
        domain_mismatch=int(domain_mismatch),
        from_return_mismatch=int(from_return_mismatch),
        has_spf=int(has_spf), has_dkim=int(has_dkim),
        num_attachments=num_attachments, has_suspicious_attachment=int(has_suspicious_attachment),
        text = fulltext  # keep text for vectorization
    )
    feats.update(kw_counts)
    return feats

# apply to DataFrame (can be slow; use .apply with progress)
from tqdm import tqdm
rows = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    rows.append(extract_features_row(row))
feats_df = pd.DataFrame(rows)
final_df = pd.concat([df.reset_index(drop=True), feats_df.reset_index(drop=True)], axis=1)
final_df.head(3800)


  0%|          | 0/4930 [00:00<?, ?it/s]C:\Users\froze\AppData\Local\Temp\ipykernel_20968\1811965851.py:19: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  return tldextract.extract(u).registered_domain
C:\Users\froze\AppData\Local\Temp\ipykernel_20968\1811965851.py:15: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  return tldextract.extract(dom).registered_domain or dom
  1%|▏         | 66/4930 [00:00<00:16, 300.82it/s]C:\Users\froze\AppData\Local\Temp\ipykernel_20968\1811965851.py:61: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has t

,subject,from,return_path,date,body_text,body_html,urls,anchors,attachments,label,...,kw_urgent,kw_click here,kw_bank,kw_login,kw_update,kw_suspend,kw_confirm,kw_security,kw_ssn,kw_credit card
0,Re: New Sequences Window,Robert Elz <kre@munnari.OZ.AU>,<exmh-workers-admin@example.com>,None,"Date: Wed, 21 Aug 2002 10:54:46 -05...",,[https://listman.redhat.com/mailman/listinfo/e...,[],[],0,...,0,0,0,0,0,0,0,0,0,0
1,[zzzzteana] RE: Alexander,Steve Burt <Steve_Burt@cursor-system.com>,<Steve_Burt@cursor-system.com>,None,"Martin A posted:\nTassos Papadopoulos, the Gre...",,[http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HA...,[],[],0,...,0,0,0,0,0,0,0,0,0,0
2,[zzzzteana] Moscow bomber,Tim Chapman <timc@2ubh.com>,<timc@2ubh.com>,None,Man Threatens Explosion In Moscow \n\nThursday...,,[http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HA...,[],[],0,...,0,0,0,0,0,0,0,5,0,0
3,[IRR] Klez: The Virus That Won't Die,Monty Solomon <monty@roscom.com>,<irregulars-admin@tb.tf>,None,Klez: The Virus That Won't Die\n \nAlready the...,,"[http://www.pcworld.com/news/article/0,aid,103...",[],[],0,...,0,0,0,0,0,0,0,1,0,0
4,Re: Insert signature,Tony Nugent <tony@linuxworks.com.au>,<exmh-users-admin@example.com>,None,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot...",,[https://listman.redhat.com/mailman/listinfo/e...,[],[],0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,Renewal Account,Commonwealth Renewal <renewal@commbank.com.au>,<nobody@galaxy.frogee.com>,None,,\n<HTML>\n<HEAD>\n<TITLE>Commonwealth Bank of ...,[http://customer-148-235-82-226.uninet-ide.com...,[(http://customer-148-235-82-226.uninet-ide.co...,[],1,...,0,0,0,0,0,0,0,0,0,0
3796,Chase Bank Online� Department Notice,"""Chase Bank Online� Department <service@chase....",<nobody@server1.websbesthosting.com>,None,,"<html>\n<xbody>\n<img src=""http://www.chase.co...",[http://chaseonline.chase.com.chaseonline.8k.c...,[(http://chaseonline.chase.com.chaseonline.8k....,[],1,...,0,0,1,0,0,0,0,0,0,0
3797,You have successfully added a new email addres...,PayPal <do-not-reply@paypal.com>,<do-not-reply@paypal.com>,None,,<html><head><title>PayPal</title></head><body>...,[http://rds.yahoo.com/S=44831148:D1/CS=4483114...,[(http://rds.yahoo.com/S=44831148:D1/CS=448311...,[],1,...,0,0,0,0,0,0,0,0,0,0
3798,Account Security Measures,"""service@AACreditUnion.org.com"" <service@AACre...",<root@ent-club.com>,None,,\n<html>\n\n\n\n\n\n<!-- The following is the ...,[http://svr2.wmsys.net/www.aacreditunion.org/c...,[(http://svr2.wmsys.net/www.aacreditunion.org/...,[],1,...,0,0,0,0,0,0,0,1,0,0


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

text_col = "text"
numeric_cols = [
    "subj_len","subj_words","body_len","body_words","uppercase_ratio",
    "exclaim_count","question_count","num_urls","num_ip_urls","num_shorteners",
    "avg_url_len","num_unique_domains","domain_mismatch","from_return_mismatch",
    "has_spf","has_dkim","num_attachments","has_suspicious_attachment"
] + [f"kw_{k}" for k in KEYWORDS]

# fill missing numeric cols (if any) and to numeric
final_df[numeric_cols] = final_df[numeric_cols].fillna(0).astype(float)

# TF-IDF (word n-grams + char n-grams optional)
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2), min_df=3)
X_text = vectorizer.fit_transform(final_df[text_col].fillna(""))

scaler = StandardScaler(with_mean=False)  # with_mean False because we'll combine with sparse matrix
X_num = scaler.fit_transform(final_df[numeric_cols].values)  # dense

# combine into sparse X
X = hstack([X_text, csr_matrix(X_num)])
y = final_df['label'].values


In [9]:

from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Logistic Regression (baseline)
clf_lr = LogisticRegression(max_iter=2000, class_weight='balanced', solver='saga')
clf_lr.fit(X_train, y_train)
probs_lr = clf_lr.predict_proba(X_test)[:,1]
preds_lr = clf_lr.predict(X_test)

print("LogReg report:")
print(classification_report(y_test, preds_lr, digits=4))
print("ROC AUC:", roc_auc_score(y_test, probs_lr))

# Random Forest
clf_rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', n_jobs=-1, random_state=42)
clf_rf.fit(X_train, y_train)
preds_rf = clf_rf.predict(X_test)
probs_rf = clf_rf.predict_proba(X_test)[:,1]
print("RF report:")
print(classification_report(y_test, preds_rf, digits=4))
print("ROC AUC:", roc_auc_score(y_test, probs_rf))


from sklearn.ensemble import VotingClassifier

# Create ensemble with soft voting
ensemble = VotingClassifier(
    estimators=[
        ("lr", clf_lr),
        ("rf", clf_rf)
    ],
    voting="soft",   # uses predict_proba instead of hard class majority
    weights=[1, 2]   # you can adjust (e.g. give more weight to RF if it's better)
)

# Fit ensemble
ensemble.fit(X_train, y_train)

# Evaluate
preds_ens = ensemble.predict(X_test)
probs_ens = ensemble.predict_proba(X_test)[:, 1]

print("\nEnsemble report:")
print(classification_report(y_test, preds_ens, digits=4))
print("ROC AUC:", roc_auc_score(y_test, probs_ens))


d:\PhishGuard-AI\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogReg report:
              precision    recall  f1-score   support

           0     0.9601    0.9475    0.9538       610
           1     0.9167    0.9362    0.9263       376

    accuracy                         0.9432       986
   macro avg     0.9384    0.9419    0.9401       986
weighted avg     0.9436    0.9432    0.9433       986

ROC AUC: 0.9856949773282175
RF report:
              precision    recall  f1-score   support

           0     0.9720    0.9689    0.9704       610
           1     0.9497    0.9548    0.9523       376

    accuracy                         0.9635       986
   macro avg     0.9609    0.9618    0.9613       986
weighted avg     0.9635    0.9635    0.9635       986

ROC AUC: 0.9959321590512731


d:\PhishGuard-AI\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Ensemble report:
              precision    recall  f1-score   support

           0     0.9752    0.9656    0.9703       610
           1     0.9450    0.9601    0.9525       376

    accuracy                         0.9635       986
   macro avg     0.9601    0.9628    0.9614       986
weighted avg     0.9637    0.9635    0.9635       986

ROC AUC: 0.9963376351587024


In [10]:
# feature names
text_names = vectorizer.get_feature_names_out()
all_feature_names = list(text_names) + numeric_cols

coefs = clf_lr.coef_[0]
# top positive tokens
top_pos = sorted(zip(coefs[:len(text_names)], text_names), reverse=True)[:20]
top_neg = sorted(zip(coefs[:len(text_names)], text_names))[:20]
print("Top text tokens (phish):", top_pos[:10])
print("Top text tokens (ham):", top_neg[:10])

# numeric feature importance (approx)
num_coefs = coefs[len(text_names):]
num_feat_imp = sorted(zip(num_coefs, numeric_cols), reverse=True)
print("Top numeric features:", num_feat_imp[:10])


Top text tokens (phish): [(np.float64(2.579571989215233), 'account'), (np.float64(2.291822449792038), 'ebay'), (np.float64(2.1717944342141), 'notification'), (np.float64(1.8035026623416603), 'paypal'), (np.float64(1.4957880852576657), 'chase'), (np.float64(1.4929781397964499), 'security'), (np.float64(1.4191819497591485), 'your account'), (np.float64(1.3902492134929003), 'update'), (np.float64(1.3599307921837882), 'posible spam'), (np.float64(1.3599307921837882), 'posible')]
Top text tokens (ham): [(np.float64(-2.9964757877675403), 'the'), (np.float64(-1.8987797167165756), '2002'), (np.float64(-1.8408728178149305), 'http'), (np.float64(-1.7581372125992736), 'http www'), (np.float64(-1.733826975415234), 'www'), (np.float64(-1.6417459582387726), 'url http'), (np.float64(-1.5976377370664263), 'to'), (np.float64(-1.5687139551245934), 'url'), (np.float64(-1.5503235397520567), 'for'), (np.float64(-1.464549434184205), 'free')]
Top numeric features: [(np.float64(1.0537566049441627), 'domain_mi

In [11]:
joblib.dump({
    "vectorizer": vectorizer,
    "scaler": scaler,
    "numeric_cols": numeric_cols,
    "model": ensemble
}, "phish_detector_joblib.pkl")


['phish_detector_joblib.pkl']

In [12]:
assets = joblib.load("phish_detector_joblib.pkl")
print(type(assets["model"]))
print(assets["model"])


<class 'sklearn.ensemble._voting.VotingClassifier'>
VotingClassifier(estimators=[('lr',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=2000,
                                                 solver='saga')),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     n_estimators=200,
                                                     n_jobs=-1,
                                                     random_state=42))],
                 voting='soft', weights=[1, 2])


In [13]:
import email
from email import policy
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
from scipy.sparse import hstack, csr_matrix

def preprocess_raw_email(raw_email_str):
    """Convert raw email string to features for prediction"""
    
    # Parse the email
    try:
        msg = email.message_from_string(raw_email_str, policy=policy.default)
    except Exception as e:
        print(f"Error parsing email: {e}")
        # Create dummy message for testing
        msg = email.message.EmailMessage()
        msg['subject'] = 'Test'
        msg['from'] = 'test@example.com'
        msg.set_content(raw_email_str)
    
    # Extract basic fields
    subject = msg.get("subject", "") or ""
    from_ = msg.get("from", "") or ""
    return_path = msg.get("return-path", "") or msg.get("Return-Path", "") or ""
    
    # Extract body text
    body_text = ""
    body_html = ""
    urls = []
    attachments = []
    
    if msg.is_multipart():
        for part in msg.walk():
            ctype = part.get_content_type()
            filename = part.get_filename()
            
            if filename:
                attachments.append(filename)
            
            if ctype == "text/plain" and not filename:
                try:
                    payload = part.get_payload(decode=True)
                    if payload:
                        body_text += payload.decode('utf-8', errors='ignore') + "\n"
                except:
                    pass
            elif ctype == "text/html" and not filename:
                try:
                    payload = part.get_payload(decode=True)
                    if payload:
                        html_content = payload.decode('utf-8', errors='ignore')
                        body_html += html_content + "\n"
                        # Extract URLs from HTML
                        soup = BeautifulSoup(html_content, 'html.parser')
                        for link in soup.find_all('a', href=True):
                            urls.append(link['href'])
                except:
                    pass
    else:
        try:
            payload = msg.get_payload(decode=True)
            if payload:
                content = payload.decode('utf-8', errors='ignore')
                if msg.get_content_type() == "text/html":
                    body_html = content
                    soup = BeautifulSoup(content, 'html.parser')
                    for link in soup.find_all('a', href=True):
                        urls.append(link['href'])
                else:
                    body_text = content
        except:
            # Fallback: treat as plain text
            body_text = raw_email_str
    
    # If no structured body found, use raw email as body text
    if not body_text and not body_html:
        body_text = raw_email_str
    
    # Extract URLs from plain text using regex
    URL_REGEX = re.compile(r"https?://[^\s'\"<>]+", re.IGNORECASE)
    urls.extend(URL_REGEX.findall(body_text))
    urls = list(set(urls))  # Remove duplicates
    
    # Clean HTML to text if needed
    if body_html and not body_text:
        try:
            soup = BeautifulSoup(body_html, 'html.parser')
            body_text = soup.get_text()
        except:
            body_text = body_html
    
    # Combine text
    full_text = (subject + " " + body_text).strip()
    
    # Initialize features dictionary
    feats = {}
    
    # Text/lexical features
    feats["subj_len"] = len(subject)
    feats["subj_words"] = len(subject.split()) if subject else 0
    feats["body_len"] = len(body_text)
    feats["body_words"] = len(body_text.split()) if body_text else 0
    
    letters = sum(1 for c in full_text if c.isalpha())
    uppercase = sum(1 for c in full_text if c.isupper())
    feats["uppercase_ratio"] = uppercase / max(1, letters)
    feats["exclaim_count"] = full_text.count("!")
    feats["question_count"] = full_text.count("?")
    
    # Keyword features
    KEYWORDS = ["verify","password","account","urgent","click here","bank","login","update","suspend","confirm","security","ssn","credit card"]
    for kw in KEYWORDS:
        feats[f"kw_{kw.replace(' ', '_')}"] = full_text.lower().count(kw)
    
    # URL features
    feats["num_urls"] = len(urls)
    
    IP_IN_URL = re.compile(r"https?://(?:\d{1,3}\.){3}\d{1,3}")
    feats["num_ip_urls"] = sum(1 for u in urls if IP_IN_URL.search(u))
    
    SHORTENER_DOMAINS = {"bit.ly", "tinyurl.com", "t.co", "goo.gl", "ow.ly", "is.gd", "buff.ly"}
    feats["num_shorteners"] = sum(1 for u in urls if urlparse(u).netloc.replace("www.","") in SHORTENER_DOMAINS)
    
    feats["avg_url_len"] = np.mean([len(u) for u in urls]) if urls else 0
    
    # Domain features (simplified)
    unique_domains = set()
    for u in urls:
        try:
            domain = urlparse(u).netloc
            if domain:
                unique_domains.add(domain)
        except:
            pass
    feats["num_unique_domains"] = len(unique_domains)
    
    # Domain mismatch (simplified - just check if we have domains)
    feats["domain_mismatch"] = 0  # Simplified for testing
    
    # Header features
    def simple_domain_extract(addr):
        if not addr:
            return ""
        # Simple extraction without tldextract
        match = re.search(r"[\w\.-]+@([\w\.-]+)", addr)
        if match:
            return match.group(1).lower()
        return addr.lower().strip()
    
    from_domain = simple_domain_extract(from_)
    return_domain = simple_domain_extract(return_path)
    
    # Fix the boolean logic
    feats["from_return_mismatch"] = int(bool(from_domain and return_domain and from_domain != return_domain))
    
    # Authentication (simplified)
    feats["has_spf"] = int("spf" in full_text.lower())
    feats["has_dkim"] = int("dkim" in full_text.lower())
    
    # Attachment features
    feats["num_attachments"] = len(attachments)
    SUSPICIOUS_EXTS = {'.exe', '.scr', '.pif', '.jar', '.js', '.vbs', '.html', '.hta', '.docm', '.xlsm', '.xls', '.pptm', '.zip', '.rar'}
    feats["has_suspicious_attachment"] = int(any(
        any(att.lower().endswith(ext) for ext in SUSPICIOUS_EXTS) 
        for att in attachments
    ))
    
    # Ensure all numeric features are present
    numeric_cols = [
        "subj_len","subj_words","body_len","body_words","uppercase_ratio",
        "exclaim_count","question_count","num_urls","num_ip_urls","num_shorteners",
        "avg_url_len","num_unique_domains","domain_mismatch","from_return_mismatch",
        "has_spf","has_dkim","num_attachments","has_suspicious_attachment"
    ] + [f"kw_{k.replace(' ', '_')}" for k in KEYWORDS]
    
    # Fill missing features with 0
    for col in numeric_cols:
        if col not in feats:
            feats[col] = 0
    
    # Create feature vector
    numeric_vector = np.array([feats[col] for col in numeric_cols]).reshape(1, -1)
    
    # Transform text (assuming vectorizer and scaler are available globally)
    try:
        text_vector = vectorizer.transform([full_text])
        numeric_scaled = scaler.transform(numeric_vector)
        X = hstack([text_vector, csr_matrix(numeric_scaled)])
    except NameError:
        print("Warning: vectorizer or scaler not found. Using dummy features.")
        # Create dummy feature matrix for testing
        X = np.zeros((1, 100))
    
    return X, feats, full_text

def custom_email(raw_email):
    """Predict whether an email is phishing or benign"""
    try:
        # Extract features
        X, feats, text = preprocess_raw_email(raw_email)
        
        # Predict (assuming model is available globally)
        try:
            prediction = model.predict(X)
            probability = model.predict_proba(X)
            
            # Format results
            label = "Benign" if prediction[0] == 0 else "Phishing"
            prob_benign = probability[0][0]
            prob_phishing = probability[0][1]
            
        except NameError:
            print("Warning: model not found. Using dummy predictions.")
            # Dummy predictions for testing
            label = "Phishing" if any(kw in text.lower() for kw in ["urgent", "verify", "click here"]) else "Benign"
            prob_benign = 0.3 if label == "Phishing" else 0.8
            prob_phishing = 0.7 if label == "Phishing" else 0.2
        
        return label, prob_benign, prob_phishing, feats
        
    except Exception as e:
        print(f"Error in custom_email: {e}")
        return "Error", 0.5, 0.5, {}

# ---------- Testing ----------
emails_to_test = [
    """
    Dear customer,
    Your account has been locked due to suspicious activity.
    Please verify your login at http://secure-banking-login.com immediately.
    """,
    """
    Hi Alice,
    Just a reminder about our team meeting tomorrow at 10 AM in the office.
    Cheers,
    Bob
    """,
    """
    Congratulations! You've won a $1000 gift card.
    Click here to claim: http://free-reward-prize.info
    """,
    """
    Dear student,
    Here is the link to download the lecture notes for tomorrow's class:
    https://university.edu/resources/notes/week3.pdf
    """,
    """
    Subject: Urgent: Verify Your Account Now

Dear Customer,

We noticed unusual activity on your account. To avoid suspension please verify your credentials immediately:
https://secure.example-phish.com/verify?user=123

If you do not verify within 24 hours your account will be locked.

Sincerely,
Account Security Team
""",
"""
Subject: Important: Confirm Recent Login

Hi,

We detected a login from a new device. If this wasn't you, please change your password now:
http://update-login.example-phish.com/change-password

Regards,
Security Alerts
""",
"""
Subject: Team Meeting Tomorrow — Agenda Attached

Hi Team,

Reminder: our weekly standup is tomorrow at 10:00 AM in Conference Room B.

Agenda:
- Project status
- Blockers
- Next steps

Thanks,
Priya
""",
"""
Subject: Urgent: Verify Your Account Now

Dear Customer,

We detected unusual activity on your account. To avoid suspension, verify your identity immediately by visiting the following page: [REMOVE_LINK]. This verification must be completed within 24 hours.

Regards,
Account Security Team
""",
"""
Pranav Rh, Urgent Requirement for the role of Intern
Inbox

Naukri Campus Jobs <recommendationnc@naukri.com>
Fri 5 Sept, 13:30
to me


naukri.com
Get App
	
Your weekend opportunities are right here!

Backend Developer Intern
Quloi			2.3
	
6 months duration
Internship	

Internship - Node.js
Data Foundry			3.4
	
3 months duration
Internship	
Jobs outside your preference

Information Technology Intern
Morgan Placement	
	
3 months duration
Internship	
Are these jobs relevant?		
Yes
No

Web Developer Intern
Ebws Llp	
	
3 months duration
Internship	
View All Recommendations
Applies are a click away on the Naukri app
Available on		
Get App

Scan to download
Facebook		Twitter		Instagram
Unsubscribe				Report a Problem
You have received this mail because your e-mail ID is registered with Naukri.com. This is a system-generated e-mail regarding your Naukri account preferences, please don't reply to this message. The jobs sent in this mail have been posted by the clients of Naukri.com. And we have enabled auto-login for your convenience, you are strongly advised not to forward this email to protect your account from unauthorized access. IEIL has taken all reasonable steps to ensure that the information in this mailer is authentic. Users are advised to research bonafides of advertisers independently. Please do not pay any money to anyone who promises to find you a job. IEIL shall not have any responsibility in this regard. We recommend that you visit our Terms & Conditions and the Security Advice for more comprehensive information.
""",
"""
Subject: URGENT: Verify Your Account Immediately!
From: support@secure-example.com
Return-Path: <mailer@phish-example.com>
Reply-To: accounts@phish-example.com
Authentication-Results: mx.example.com; spf=fail smtp.mailfrom=phish-example.com; dkim=pass header.d=secure-example.com
Date: Tue, 1 Oct 2025 12:00:00 +0000
Content-Type: multipart/mixed; boundary="ABC123"

--ABC123
Content-Type: text/html; charset="utf-8"

<html>
  <body>
    <p>Dear Customer,</p>
    <p>YOUR ACCOUNT WILL BE LOCKED if you do not VERIFY your credentials immediately!</p>
    <p>Click the link below to confirm your login:</p>
    <ul>
      <li><a href="http://malicious.example-phish.com/login">www.secure-example.com</a></li>
      <li>Or use this shortcut link: http://bit.ly/fake-login</li>
      <li>IP login portal: http://192.168.10.10/verify</li>
    </ul>
    <form action="http://forms.example-phish.com/submit" method="post">
      <input type="text" name="user"/>
      <input type="password" name="pass"/>
      <button>Submit</button>
    </form>
    <script>
      window.location = "http://js-redirect.example-phish.com/steal";
    </script>
    <p>Thank you,<br/>Secure Support Team</p>
  </body>
</html>

--ABC123
Content-Disposition: attachment; filename="Invoice_999.docm"
Content-Type: application/msword
Content-Transfer-Encoding: base64

[BASE64_CONTENT_TRUNCATED]

--ABC123--
""",
"""
Subject: Urgent – Verify Your Account

Dear user,  
Your account has been flagged. Please click the link below to verify:  
http://bit.ly/fake-login  
"""




]

print("Testing Email Classification:")
print("=" * 80)

for i, raw_email in enumerate(emails_to_test, 1):
    print(f"\n--- Email {i} ---")
    label, prob_benign, prob_phishing, feats = custom_email(raw_email)
    
    print("Email snippet:", raw_email.strip()[:80] + "...")
    print(f"Prediction: {label}")
    print(f"Probability (Benign): {prob_benign:.3f}")
    print(f"Probability (Phishing): {prob_phishing:.3f}")
    
    # Show some key features
    key_features = ["num_urls", "kw_verify", "kw_urgent", "kw_click_here", "uppercase_ratio"]
    relevant_feats = {k: v for k, v in feats.items() if k in key_features and v > 0}
    if relevant_feats:
        print("Key features:", relevant_feats)
    
    print("-" * 60)

Testing Email Classification:

--- Email 1 ---
Email snippet: Dear customer,
    Your account has been locked due to suspicious activity.
    ...
Prediction: Phishing
Probability (Benign): 0.300
Probability (Phishing): 0.700
Key features: {'uppercase_ratio': 0.025423728813559324, 'kw_verify': 1, 'num_urls': 1}
------------------------------------------------------------

--- Email 2 ---
Email snippet: Hi Alice,
    Just a reminder about our team meeting tomorrow at 10 AM in the of...
Prediction: Benign
Probability (Benign): 0.800
Probability (Phishing): 0.200
Key features: {'uppercase_ratio': 0.09859154929577464}
------------------------------------------------------------

--- Email 3 ---
Email snippet: Congratulations! You've won a $1000 gift card.
    Click here to claim: http://f...
Prediction: Phishing
Probability (Benign): 0.300
Probability (Phishing): 0.700
Key features: {'uppercase_ratio': 0.04225352112676056, 'kw_click_here': 1, 'num_urls': 1}
---------------------------------

In [14]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Assuming you have X_train, y_train, X_test, y_test from your original code
# Base models
clf_lr = LogisticRegression(max_iter=2000, class_weight='balanced', solver='saga')
clf_rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', n_jobs=-1, random_state=42)

# Meta-model (a simple Logistic Regression works well)
meta_clf = LogisticRegression()

# Stacking Classifier
# You can use 'predict' or 'predict_proba' for the meta-model input.
# 'use_probas=True' often gives better results.
sclf = StackingClassifier(
    classifiers=[clf_lr, clf_rf],
    meta_classifier=meta_clf,
    use_probas=True,
    passthrough=True, # Pass the original features to the meta-model as well
    verbose=2
)

sclf.fit(X_train, y_train)
stacked_preds = sclf.predict(X_test)
stacked_probs = sclf.predict_proba(X_test)[:, 1]

print("Stacked Ensemble Report:")
print(classification_report(y_test, stacked_preds, digits=4))
print("ROC AUC:", roc_auc_score(y_test, stacked_probs))

TypeError: StackingClassifier.__init__() got an unexpected keyword argument 'passthrough'